# DC Line dispatch with pandapower OPF
This is an introduction into the usage of the pandapower optimal power flow with dc lines.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf_dcline.png" width="100%">

We first create this network in pandapower:

In [66]:
import pandapower as pp
from numpy import array
net = pp.create_empty_network()

b1 = pp.create_bus(net, 380)
b2 = pp.create_bus(net, 380)
b3 = pp.create_bus(net, 380)
b4 = pp.create_bus(net, 380)
b5 = pp.create_bus(net, 380)

l1 = pp.create_line(net, b1, b2, 30, "490-AL1/64-ST1A 380.0")
l2 = pp.create_line(net, b3, b4, 20, "490-AL1/64-ST1A 380.0")
l3 = pp.create_line(net, b4, b5, 20, "490-AL1/64-ST1A 380.0")

dcl1 = pp.create_dcline(net, name="dc line", from_bus=b2, to_bus=b3, p_kw=0.2e6, loss_percent=1.0, 
                  loss_kw=500, vm_from_pu=1.01, vm_to_pu=1.012, max_p_kw=1e6,
                  in_service=True)

eg1 = pp.create_ext_grid(net, b1, 1.02, max_p_kw=0.)
eg2 = pp.create_ext_grid(net, b5, 1.02, max_p_kw=0.)

l1 = pp.create_load(net, bus=b4, p_kw=800e3, controllable = False)

We now run a regular load flow to check out the DC line model:

In [67]:
pp.runpp(net)

The transmission power of the DC line is defined in the loadflow as given by the p_kw parameter, which was set to 0.2 GW:

In [68]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,200000.0,152443.185449,-197500.0,74491.758943,2500.0,1.01,-0.48595,1.012,-0.725627


The losses amount to 2500 kW, which are made up of 500 kW conversion loss and 200 MW * 0.01 = 2 MW transmission losses. The voltage setpoints defined at from and to bus are complied with. 

Now lets define costs for the external grids to run an OPF:

In [69]:
costeg0 = pp.create_polynomial_cost(net, 0, 'ext_grid', array([-.1, 0]))
costeg1 = pp.create_polynomial_cost(net, 1, 'ext_grid', array([-.08, 0]))
net.bus['max_vm_pu'] = 1.5
net.line['max_loading_percent'] = 1000


In [70]:
pp.runopp(net)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


Since we defined lower costs for Ext Grid 2, it fully services the load:

In [71]:
net.res_ext_grid

,p_kw,q_kvar
0,-821525.329085,7787.604746
1,-0.082348,21048.664329


While the DC line does not transmit any power:

In [72]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,813573.855039,-26446.091117,-805023.618851,-21736.382564,8550.236189,1.011014,-2.39987,1.027504,1.522333


If we set the costs of the left grid to a lower value than the right grid and run the loadflow again:

In [73]:
net.polynomial_cost.c.at[costeg0]= array([[-0.08, 0]])
net.polynomial_cost.c.at[costeg1]= array([[-0.1, 0]])
pp.runopp(net)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


We can see that the power now comes from the left ext_grid:

In [74]:
net.res_ext_grid

,p_kw,q_kvar
0,-500.045496,7787.585468
1,-805091.509265,-628.276142


And is transmitted over the DC line:

In [75]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,500.04255,7787.428299,-0.042128,-627.159257,500.000421,1.019994,-0.001447,1.013925,-1.563437


We can however see that the lines on the left hand side are now overloaded:

In [76]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,500.045496,-7787.585468,-500.042550,-7787.428299,0.002946,-15575.013767,0.011624,0.011624,0.011624,1.210823
1,0.042128,627.159257,0.221339,-10884.114753,0.263467,-10256.955496,0.000940,0.016313,0.016313,1.699249
2,-800000.221339,10884.114753,805091.509265,628.276142,5091.287926,11512.390896,1.199128,1.199226,1.199226,124.919360


If we set the maximum line loading to 100% and run the OPF again:

In [77]:
net.line["max_loading_percent"] = 100
pp.runopp(net)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


We can see that the lines are no longer overloaded:

In [78]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,161369.601227,-7787.628273,-161062.805588,-6442.842973,306.795638,-14230.471246,0.240648,0.240553,0.240648,25.067513
1,158973.069027,-5711.282156,-158773.057286,-3723.513123,200.011741,-9434.795279,0.237850,0.237754,0.237850,24.776062
2,-641226.930948,3723.511692,644489.605777,-64.356230,3262.674829,3659.155462,0.959956,0.960001,0.960001,100.000067


Because the load is serviced from both grids:

In [79]:
net.res_ext_grid

,p_kw,q_kvar
0,-161369.601227,7787.628273
1,-644489.601030,64.355742


And the DC line transmits only part of the power needed to service the load:

In [80]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,161062.805588,6442.842974,-158973.07484,5711.329933,2089.730748,1.018095,-0.467989,1.016146,-0.937997


Finally, we can also define transmission costs for the DC line:

In [81]:
costeg1 = pp.create_polynomial_cost(net, 0, 'dcline', array([.03, 0]))
pp.runopp(net)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


Because the sum of the costs for generating power on the left hand side (0.08) and transmitting it to the right side (0.03) is now larger than for generating on the right side (0.1), the OPF draws as much power from the right side as is possible without violating line loading constraints:

In [82]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,161369.750347,-7787.632437,-161062.954141,-6442.836355,306.796205,-14230.468792,0.240648,0.240553,0.240648,25.067536
1,158973.218473,-5627.157184,-158773.209333,-3807.737669,200.009141,-9434.894853,0.237845,0.237757,0.237845,24.775477
2,-641226.783716,3807.736825,644489.450237,-148.646474,3262.666521,3659.090352,0.959953,0.960000,0.960000,100.000045


In [83]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,161062.954141,6442.836355,-158973.221922,5627.185349,2089.732219,1.018095,-0.467989,1.016151,-0.938072


If we broaden the line loading constraint and run the OPF again:

In [84]:
net.line["max_loading_percent"] = 1000
pp.runopp(net)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['dcline']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.


The load is once again fully serviced by the grid on the right hand side:

In [85]:
net.res_ext_grid

,p_kw,q_kvar
0,-500.027328,7787.596407
1,-805091.527483,-628.264114


And the DC line is in open loop operation:

In [86]:
net.res_dcline

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree
0,500.024382,7787.417373,-0.024141,-627.172262,500.000241,1.019994,-0.001447,1.013925,-1.563437
